### Import files

In [18]:
import numpy as np
import scipy as sp
import pandas as pd
import json

data_loc = "../../data/intact/"

In [31]:
spA = "human"

dfA = pd.read_csv(data_loc + spA + ".s.tsv", sep = "\t", header = None)

nodesA = set(dfA[0]).union(dfA[1])

nA = {k: i for i, k in enumerate(nodesA)}

with open(f"temp-data/{spA}.json", "w") as of:
    json.dump(nA, of)

udfA = dfA.copy()

udfA.loc[:, 0] = udfA.loc[:, 0].apply(lambda x : nA[x])
udfA.loc[:, 1] = udfA.loc[:, 1].apply(lambda x : nA[x])

/tmp/ipykernel_13274/3359673858.py:14: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  udfA.loc[:, 0] = udfA.loc[:, 0].apply(lambda x : nA[x])
/tmp/ipykernel_13274/3359673858.py:15: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  udfA.loc[:, 1] = udfA.loc[:, 1].apply(lambda x : nA[x])


In [32]:
from scipy.sparse import coo_array

A_x = np.concatenate([udfA[0].values, udfA[1].values]) 
A_y = np.concatenate([udfA[1].values, udfA[0].values]) 

A = coo_array((np.ones(A_x.shape), (A_x, A_y)), shape = (len(nA), len(nA)))

In [33]:
from scipy.io import savemat
savemat(f"temp-data/{spA}.mat", {f"{spA}" : A})

## Save pair files

In [34]:
import os

data_loc = "../../data/intact/"

In [36]:
species = ["human", "mouse", "rat", "fly", "bakers"]


for i in range(5):
    for j in range(i):
        spA = species[i]
        spB = species[j]

        with open(f"temp-data/{spA}.json", "r") as jA, open(f"temp-data/{spB}.json", "r") as jB: 
            nA = json.load(jA)
            nB = json.load(jB)
        efile = data_loc + f"{spA}-{spB}.tsv"
        if not os.path.exists(efile):
            efile = data_loc + f"{spB}-{spA}.tsv"
        dfm = pd.read_csv(efile, sep = "\t")
        dfm = dfm.loc[:, [spA, spB, "score"]]
        dfm[spA] = dfm[spA].apply(lambda x: nA[x])
        dfm[spB] = dfm[spB].apply(lambda x: nB[x])

        iloc = dfm[spA].values
        jloc = dfm[spB].values

        score = dfm["score"].values

        E = coo_array((score, (iloc, jloc)), size = (len(nA), len(nB)))
        savemat(f"temp-data/{spA}-{spB}.mat", {f"{spA}-{spB}" : E})

SyntaxError: invalid syntax (4158463558.py, line 26)